* changed by nov05 on 2024-11-28  

* Udacity AWS MLE Nanodegree (ND189)  
  Course 4, 3.15 Excercise: Fine-Tuning BERT    
  
* ~~local env `conda activate cuda_py310` with cuda enabled~~  
  moved to `google colab` for traning on better GPU   
  
* [CoLA dataset on KaggleHub](https://www.kaggle.com/datasets/krazy47/cola-the-corpus-of-linguistic-acceptability)  
  The Corpus of Linguistic Acceptability   

* [baseline notebook in colab](https://drive.google.com/file/d/11V8yjHs5rna2Kg8DHMhnm3XEBbR9m2mY)  
  [W&B training logs](https://wandb.ai/nov05/udacity-awsmle-bert-cola/runs/3blkneau)   
  Run summary:  
  ```
  eval_accuracy_epoch (%)	81.44089
  train_loss	0.00013
  train_loss_epoch	0.02814
  ```

  <img src="https://raw.githubusercontent.com/nov05/pictures/refs/heads/master/Udacity/20241119_aws-mle-nanodegree/2024-11-29%2007_33_08-solar-thunder-20-baseline%20_%20udacity-awsmle-bert-cola%20%E2%80%93%20Weights%20%26%20Biases.jpg" width=600>